# Minimization of metabolic adjustment analysis

TODO MOMA citation

In [1]:
import Downloads: download

!isfile("e_coli_core.json") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.json", "e_coli_core.json")

using COBREXA
import AbstractFBCModels.CanonicalModel as CM
import JSONFBCModels
import Clarabel

TODO this might do the convert immediately as with the old cobrexa...
probably better have an actual output-type argument tho rather than force the
guessing.

In [2]:
model = convert(CM.Model, load_model("e_coli_core.json"))

reference_fluxes =
    parsimonious_flux_balance_analysis(
        model,
        Clarabel.Optimizer,
        settings = [silence],
    ).fluxes

ConstraintTrees.Tree{Float64} with 95 elements:
  :ACALD                    => -1.10005e-10
  :ACALDt                   => -3.18304e-11
  :ACKr                     => -4.99126e-11
  :ACONTa                   => 6.00725
  :ACONTb                   => 6.00725
  :ACt2r                    => -3.36149e-11
  :ADK1                     => -9.84708e-12
  :AKGDH                    => 5.06438
  :AKGt2r                   => -4.87389e-11
  :ALCD2x                   => -6.24252e-11
  :ATPM                     => 8.39
  :ATPS4r                   => 45.514
  :BIOMASS_Ecoli_core_w_GAM => 0.873922
  :CO2t                     => -22.8098
  :CS                       => 6.00725
  :CYTBD                    => 43.599
  :D_LACt2                  => -4.4392e-11
  :ENO                      => 14.7161
  :ETOHt2r                  => -4.18485e-11
  ⋮                         => ⋮

TODO MOMA from here

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*